In [69]:
!pip install gcloud

In [70]:
!gcloud auth application-default login

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=EkiGbJcxmrM5JRt4wyBpotjW4k6qtd&prompt=consent&token_usage=remote&access_type=offline&code_challenge=aVaj-x5Cs3n0fuMM1EamLKBpSgbqWlOH2EOey4q9NYs&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AcvDMrCVb1MtTd7LwTP6NHRE2Ql8uT9fmED3G26pEoAtqiauIyKq7FCEjznnCc7neRWn8w

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Ca

In [71]:
##importando pacotes
import pandas as pd
import numpy as np
import time
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import openpyxl
import csv
import re

In [72]:
client = bigquery.Client(project='repositoriodedadosgpsp')
query = "SELECT ano, tipo_ocupacao,sigla_uf,id_municipio, sum(n_profissionais) as quantidade_vinculos  FROM `repositoriodedadosgpsp.Datalake.CNES_vinculo_saude_tipo_ocupacao_v2` where publico_privado='Público' group by ano, tipo_ocupacao, sigla_uf,id_municipio"
df = client.query(query).to_dataframe()
df

/usr/local/lib/python3.10/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


,ano,tipo_ocupacao,sigla_uf,id_municipio,quantidade_vinculos
0,2008,Técnicos de laboratórios de saúde e bancos de ...,AP,1600535,1
1,2008,Trabalhadores em serviços de promoção e apoio ...,AP,1600709,47
2,2008,Tecnólogos e técnicos em métodos de diagnóstic...,DF,None,270
3,2008,Assistentes sociais e economistas domésticos,AP,1600600,5
4,2008,Trabalhadores em serviços de promoção e apoio ...,AP,1600055,14
...,...,...,...,...,...
1836885,2024,Técnicos e auxiliares de enfermagem,TO,1712009,15
1836886,2024,Engenheiros civis e afins,TO,1721000,2
1836887,2024,Biomédicos,TO,1712157,1
1836888,2024,Gestores e especialistas de operações em empre...,TO,1700301,2


In [73]:
df['tipo_2'] = np.where(df['tipo_ocupacao'].str.contains('Assistentes sociais e economistas domésticos'),'Assistentes sociais e economistas domésticos',df['tipo_ocupacao'])
df

,ano,tipo_ocupacao,sigla_uf,id_municipio,quantidade_vinculos,tipo_2
0,2008,Técnicos de laboratórios de saúde e bancos de ...,AP,1600535,1,Técnicos de laboratórios de saúde e bancos de ...
1,2008,Trabalhadores em serviços de promoção e apoio ...,AP,1600709,47,Trabalhadores em serviços de promoção e apoio ...
2,2008,Tecnólogos e técnicos em métodos de diagnóstic...,DF,None,270,Tecnólogos e técnicos em métodos de diagnóstic...
3,2008,Assistentes sociais e economistas domésticos,AP,1600600,5,Assistentes sociais e economistas domésticos
4,2008,Trabalhadores em serviços de promoção e apoio ...,AP,1600055,14,Trabalhadores em serviços de promoção e apoio ...
...,...,...,...,...,...,...
1836885,2024,Técnicos e auxiliares de enfermagem,TO,1712009,15,Técnicos e auxiliares de enfermagem
1836886,2024,Engenheiros civis e afins,TO,1721000,2,Engenheiros civis e afins
1836887,2024,Biomédicos,TO,1712157,1,Biomédicos
1836888,2024,Gestores e especialistas de operações em empre...,TO,1700301,2,Gestores e especialistas de operações em empre...


In [75]:
df = df[(df['tipo_2']=='Assistentes sociais e economistas domésticos') & (df['ano']>2017) & (df['ano'] <= 2023)]
df

,ano,tipo_ocupacao,sigla_uf,id_municipio,quantidade_vinculos,tipo_2
764908,2018,Assistentes sociais e economistas domésticos,AP,1600600,18,Assistentes sociais e economistas domésticos
764946,2018,Assistentes sociais e economistas domésticos,AC,1200336,3,Assistentes sociais e economistas domésticos
764982,2018,Assistentes sociais e economistas domésticos,AC,1200401,56,Assistentes sociais e economistas domésticos
765021,2018,Assistentes sociais e economistas domésticos,AL,2708956,1,Assistentes sociais e economistas domésticos
765034,2018,Assistentes sociais e economistas domésticos,AL,2702405,12,Assistentes sociais e economistas domésticos
...,...,...,...,...,...,...
1787012,2023,Assistentes sociais e economistas domésticos,TO,1705508,8,Assistentes sociais e economistas domésticos
1787026,2023,Assistentes sociais e economistas domésticos,TO,1703305,1,Assistentes sociais e economistas domésticos
1787035,2023,Assistentes sociais e economistas domésticos,TO,1718501,1,Assistentes sociais e economistas domésticos
1787047,2023,Assistentes sociais e economistas domésticos,TO,1718865,1,Assistentes sociais e economistas domésticos


In [76]:
## Chamando a quantidade de população.

pop = pd.read_csv("POP_CD2022_ATT.csv");
pop

,UF,COD. UF,COD. MUNIC,NOME DO MUNICÍPIO,POP. COLETADA,POP. IMPUTADA,POPULAÇÃO,id_municipio
0,RO,11,15,Alta Floresta D'Oeste,20865,629,21494,1100015
1,RO,11,23,Ariquemes,93006,3827,96833,1100023
2,RO,11,31,Cabixi,5100,251,5351,1100031
3,RO,11,49,Cacoal,84923,1964,86887,1100049
4,RO,11,56,Cerejeiras,15297,593,15890,1100056
...,...,...,...,...,...,...,...,...
5565,GO,52,22005,Vianópolis,14345,611,14956,5222005
5566,GO,52,22054,Vicentinópolis,8592,176,8768,5222054
5567,GO,52,22203,Vila Boa,4168,47,4215,5222203
5568,GO,52,22302,Vila Propício,5594,221,5815,5222302


In [77]:
df['id_municipio'] = pd.to_numeric(df['id_municipio'])
df

<ipython-input-77-10d4f6141bff>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['id_municipio'] = pd.to_numeric(df['id_municipio'])


,ano,tipo_ocupacao,sigla_uf,id_municipio,quantidade_vinculos,tipo_2
764908,2018,Assistentes sociais e economistas domésticos,AP,1600600,18,Assistentes sociais e economistas domésticos
764946,2018,Assistentes sociais e economistas domésticos,AC,1200336,3,Assistentes sociais e economistas domésticos
764982,2018,Assistentes sociais e economistas domésticos,AC,1200401,56,Assistentes sociais e economistas domésticos
765021,2018,Assistentes sociais e economistas domésticos,AL,2708956,1,Assistentes sociais e economistas domésticos
765034,2018,Assistentes sociais e economistas domésticos,AL,2702405,12,Assistentes sociais e economistas domésticos
...,...,...,...,...,...,...
1787012,2023,Assistentes sociais e economistas domésticos,TO,1705508,8,Assistentes sociais e economistas domésticos
1787026,2023,Assistentes sociais e economistas domésticos,TO,1703305,1,Assistentes sociais e economistas domésticos
1787035,2023,Assistentes sociais e economistas domésticos,TO,1718501,1,Assistentes sociais e economistas domésticos
1787047,2023,Assistentes sociais e economistas domésticos,TO,1718865,1,Assistentes sociais e economistas domésticos


In [78]:
x=df.pivot_table(index=['ano','sigla_uf'  ,'id_municipio','tipo_2'], values='quantidade_vinculos',aggfunc=np.sum)
y = pd.DataFrame(x)
df = y.reset_index()
df

<ipython-input-78-35f37a736691>:1: FutureWarning: The provided callable <function sum at 0x78276451add0> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  x=df.pivot_table(index=['ano','sigla_uf'  ,'id_municipio','tipo_2'], values='quantidade_vinculos',aggfunc=np.sum)


,ano,sigla_uf,id_municipio,tipo_2,quantidade_vinculos
0,2018,AC,1200013,Assistentes sociais e economistas domésticos,1
1,2018,AC,1200054,Assistentes sociais e economistas domésticos,1
2,2018,AC,1200104,Assistentes sociais e economistas domésticos,2
3,2018,AC,1200179,Assistentes sociais e economistas domésticos,1
4,2018,AC,1200203,Assistentes sociais e economistas domésticos,21
...,...,...,...,...,...
24220,2023,TO,1721109,Assistentes sociais e economistas domésticos,1
24221,2023,TO,1721208,Assistentes sociais e economistas domésticos,5
24222,2023,TO,1721257,Assistentes sociais e economistas domésticos,1
24223,2023,TO,1722081,Assistentes sociais e economistas domésticos,2


In [79]:
print(pop.columns);
print(type(pop));

Index(['UF', 'COD. UF', 'COD. MUNIC', 'NOME DO MUNICÍPIO', 'POP. COLETADA',
       'POP. IMPUTADA', 'POPULAÇÃO', 'id_municipio'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>


In [80]:
df1= df.merge(pop[['UF','NOME DO MUNICÍPIO','id_municipio','POPULAÇÃO']], left_on='id_municipio',right_on='id_municipio').sort_values('quantidade_vinculos', ascending=False)

In [81]:
df1

,ano,sigla_uf,id_municipio,tipo_2,quantidade_vinculos,UF,NOME DO MUNICÍPIO,POPULAÇÃO
21398,2022,SP,3550308,Assistentes sociais e economistas domésticos,2022,SP,São Paulo,11451999
21399,2023,SP,3550308,Assistentes sociais e economistas domésticos,1992,SP,São Paulo,11451999
21394,2018,SP,3550308,Assistentes sociais e economistas domésticos,1974,SP,São Paulo,11451999
21397,2021,SP,3550308,Assistentes sociais e economistas domésticos,1953,SP,São Paulo,11451999
21395,2019,SP,3550308,Assistentes sociais e economistas domésticos,1875,SP,São Paulo,11451999
...,...,...,...,...,...,...,...,...
12949,2021,PR,4101150,Assistentes sociais e economistas domésticos,1,PR,Ângulo,3235
12950,2022,PR,4101150,Assistentes sociais e economistas domésticos,1,PR,Ângulo,3235
12951,2023,PR,4101150,Assistentes sociais e economistas domésticos,1,PR,Ângulo,3235
12953,2019,PR,4101200,Assistentes sociais e economistas domésticos,1,PR,Antonina,18091


In [82]:
df1['taxa'] = df1['quantidade_vinculos']/df1['POPULAÇÃO']*1000

In [83]:
df1

,ano,sigla_uf,id_municipio,tipo_2,quantidade_vinculos,UF,NOME DO MUNICÍPIO,POPULAÇÃO,taxa
21398,2022,SP,3550308,Assistentes sociais e economistas domésticos,2022,SP,São Paulo,11451999,0.176563
21399,2023,SP,3550308,Assistentes sociais e economistas domésticos,1992,SP,São Paulo,11451999,0.173943
21394,2018,SP,3550308,Assistentes sociais e economistas domésticos,1974,SP,São Paulo,11451999,0.172372
21397,2021,SP,3550308,Assistentes sociais e economistas domésticos,1953,SP,São Paulo,11451999,0.170538
21395,2019,SP,3550308,Assistentes sociais e economistas domésticos,1875,SP,São Paulo,11451999,0.163727
...,...,...,...,...,...,...,...,...,...
12949,2021,PR,4101150,Assistentes sociais e economistas domésticos,1,PR,Ângulo,3235,0.309119
12950,2022,PR,4101150,Assistentes sociais e economistas domésticos,1,PR,Ângulo,3235,0.309119
12951,2023,PR,4101150,Assistentes sociais e economistas domésticos,1,PR,Ângulo,3235,0.309119
12953,2019,PR,4101200,Assistentes sociais e economistas domésticos,1,PR,Antonina,18091,0.055276


In [84]:
sum(df1[df1['ano']==2023]['POPULAÇÃO'])

193868244

In [85]:
sum(df1[df1['ano']==2023]['quantidade_vinculos'])/sum(df1[df1['ano']==2023]['POPULAÇÃO'])*1000

0.19892375978811672

In [86]:
## validando alguns dados
df1[(df1['ano']==2023) & (df1['UF']=='RJ')].sort_values('taxa', ascending=False).head(30)

,ano,sigla_uf,id_municipio,tipo_2,quantidade_vinculos,UF,NOME DO MUNICÍPIO,POPULAÇÃO,taxa
14375,2023,RJ,3300936,Assistentes sociais e economistas domésticos,12,RJ,Carapebus,13847,0.866614
14679,2023,RJ,3304508,Assistentes sociais e economistas domésticos,6,RJ,Rio das Flores,8954,0.670092
14733,2023,RJ,3305133,Assistentes sociais e economistas domésticos,4,RJ,São José de Ubá,7070,0.565771
14775,2023,RJ,3305604,Assistentes sociais e economistas domésticos,12,RJ,Silva Jardim,21352,0.562008
14535,2023,RJ,3302809,Assistentes sociais e economistas domésticos,9,RJ,Mendes,17502,0.514227
14451,2023,RJ,3301876,Assistentes sociais e economistas domésticos,14,RJ,Iguaba Grande,27920,0.501433
14403,2023,RJ,3301207,Assistentes sociais e economistas domésticos,8,RJ,Carmo,17198,0.46517
14505,2023,RJ,3302403,Assistentes sociais e economistas domésticos,114,RJ,Macaé,246391,0.462679
14661,2023,RJ,3304201,Assistentes sociais e economistas domésticos,59,RJ,Resende,129612,0.455205
14351,2023,RJ,3300605,Assistentes sociais e economistas domésticos,15,RJ,Bom Jesus do Itabapoana,35173,0.426463


In [87]:
df1.columns

Index(['ano', 'sigla_uf', 'id_municipio', 'tipo_2', 'quantidade_vinculos',
       'UF', 'NOME DO MUNICÍPIO', 'POPULAÇÃO', 'taxa'],
      dtype='object')

In [88]:
## Renomear colunas

df = df1[['ano', 'sigla_uf', 'id_municipio', 'tipo_2', 'quantidade_vinculos',
        'NOME DO MUNICÍPIO', 'POPULAÇÃO', 'taxa']].rename(columns={'tipo_2':'tipo_ocupacao','UF':'sigla_uf', 'NOME DO MUNICÍPIO':'nome_municipio','POPULAÇÃO':'populacao_domiciliada' })

In [89]:
df.columns

Index(['ano', 'sigla_uf', 'id_municipio', 'tipo_ocupacao',
       'quantidade_vinculos', 'nome_municipio', 'populacao_domiciliada',
       'taxa'],
      dtype='object')

In [90]:
df= df[['ano', 'sigla_uf', 'nome_municipio','id_municipio','populacao_domiciliada', 'tipo_ocupacao',
       'quantidade_vinculos',
       'taxa']]

In [91]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24225 entries, 21398 to 24224
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ano                    24225 non-null  Int64  
 1   sigla_uf               24225 non-null  object 
 2   nome_municipio         24225 non-null  object 
 3   id_municipio           24225 non-null  int64  
 4   populacao_domiciliada  24225 non-null  int64  
 5   tipo_ocupacao          24225 non-null  object 
 6   quantidade_vinculos    24225 non-null  Int64  
 7   taxa                   24225 non-null  Float64
dtypes: Float64(1), Int64(2), int64(2), object(3)
memory usage: 1.7+ MB


In [97]:
df

,ano,sigla_uf,nome_municipio,id_municipio,populacao_domiciliada,tipo_ocupacao,quantidade_vinculos,taxa
21398,2022,SP,São Paulo,3550308,11451999,Assistentes sociais e economistas domésticos,2022,0.176563
21399,2023,SP,São Paulo,3550308,11451999,Assistentes sociais e economistas domésticos,1992,0.173943
21394,2018,SP,São Paulo,3550308,11451999,Assistentes sociais e economistas domésticos,1974,0.172372
21397,2021,SP,São Paulo,3550308,11451999,Assistentes sociais e economistas domésticos,1953,0.170538
21395,2019,SP,São Paulo,3550308,11451999,Assistentes sociais e economistas domésticos,1875,0.163727
...,...,...,...,...,...,...,...,...
12949,2021,PR,Ângulo,4101150,3235,Assistentes sociais e economistas domésticos,1,0.309119
12950,2022,PR,Ângulo,4101150,3235,Assistentes sociais e economistas domésticos,1,0.309119
12951,2023,PR,Ângulo,4101150,3235,Assistentes sociais e economistas domésticos,1,0.309119
12953,2019,PR,Antonina,4101200,18091,Assistentes sociais e economistas domésticos,1,0.055276


In [99]:
# Selecionar e imprimir as informações das linhas onde id_municipio é 3550308
print(df[df['id_municipio'] == 2202091])


        ano sigla_uf             nome_municipio  id_municipio  \
12452  2018       PI  Caldeirão Grande do Piauí       2202091   
12453  2019       PI  Caldeirão Grande do Piauí       2202091   
12454  2020       PI  Caldeirão Grande do Piauí       2202091   
12455  2021       PI  Caldeirão Grande do Piauí       2202091   
12456  2022       PI  Caldeirão Grande do Piauí       2202091   
12457  2023       PI  Caldeirão Grande do Piauí       2202091   

       populacao_domiciliada                                 tipo_ocupacao  \
12452                   5503  Assistentes sociais e economistas domésticos   
12453                   5503  Assistentes sociais e economistas domésticos   
12454                   5503  Assistentes sociais e economistas domésticos   
12455                   5503  Assistentes sociais e economistas domésticos   
12456                   5503  Assistentes sociais e economistas domésticos   
12457                   5503  Assistentes sociais e economistas domésticos  

Subindo para o GBQ

In [100]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de referencia da informacao'),
 bigquery.SchemaField('sigla_uf','STRING',description='Sigla da Unidade da Federação.'),
 bigquery.SchemaField('nome_municipio','STRING',description='Nome do município da observação'),
 bigquery.SchemaField('id_municipio','FLOAT',description='Identificador do município pelo IBGE'),
 bigquery.SchemaField('populacao_domiciliada','FLOAT',description='População domiciliada (prévia do Censo 2022)'),
 bigquery.SchemaField('tipo_ocupacao','STRING',description='Qual a ocupação daquele vínculo'),
 bigquery.SchemaField('quantidade_vinculos','INTEGER',description='Quantidade de vinculos'),
 bigquery.SchemaField('taxa','Float',description='Taxa de Assistentes sociais e economistas domésticos por 1000 habitantes')
 ]

In [101]:
## Subindo para datalake
client = bigquery.Client(project='repositoriodedadosgpsp')
dataset_ref = client.dataset('perfil_remuneracao')

/usr/local/lib/python3.10/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [102]:
table_ref = dataset_ref.table('CNES_assistentes_sociais_mil_habitantes_v2') # nome da tabela no padrão FONTE_algo_intuitivo_dado
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result()

LoadJob<project=repositoriodedadosgpsp, location=US, id=7210a672-2ea7-4ae7-af7f-f7a273ed610b>